In [ ]:
# Create and train the MPS
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False

# Training parameters
gn_var = 0.3 #added gaussian noise variance
gn_mean = 0 #added gaussian noise mean
n_epochs_lmps = 18
mps_learn_rate = 0.0001
mps_reg = 0.0
mps_chosen_loss = nn.CrossEntropyLoss().to(chosen_device)


# Initialise the local_patch MPS
# Create the fcnn class
class LMPS_patcher(nn.Module):
    def __init__(self):
        super(LMPS_patcher, self).__init__()
        mps_student = MPS(
            input_dim=28 ** 2,
            output_dim=10,
            bond_dim=bond_dim,
            adaptive_mode=adaptive_mode,
            periodic_bc=periodic_bc,
        )

    def forward(self, mps_student):
        y = self.mps(x)
        return y

student = LMPS_patcher()

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(mps_student.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)
softmax = nn.Softmax(dim=1)

# Training loop 
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape, add gaussian noise, put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        x_mb = (x_mb + torch.randn(size=x_mb.size())).to(chosen_device)

        # Get softmax of output of teacher
        y_mb = (softmax(fcnn_teacher(x_mb))).to(chosen_device)

        # Foward propagation
        y_hat_mb = mps_student(x_mb)
        loss = mps_chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps_optimizer.step()
        lmps_optimizer.zero_grad()

    print(loss.item())




# get accuracy
for (x_mb, y_mb) in test_iterator:
    print("testing teacher")
    print(MulticlassAccuracy(num_classes=10)(mps_student(x_mb.reshape(-1, 784)), y_mb))
